![Pandas logo](img/pandas.svg)

# Reading and Writing Data

Pandas comes with I/O functions for a great many different formats.  You can get a feel for most of them with the Jupyter interactive help, by typing `pd.read_<tab>` to get a pop-up showing the functions with that prefix.  You can get a similar list of built-in output formats once you have loaded a DataFrame by typing `df.to_<tab>`.

A partial list of formats Pandas supports are:

- [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
- [Excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html)
- [SQL](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html)
- [JSON](http://www.json.org/)
- [HDF5](https://www.hdfgroup.org/HDF5/)
- [pickle](https://docs.python.org/3/library/pickle.html)
- [msgpack](http://msgpack.org/)
- [Stata](https://en.wikipedia.org/wiki/Stata)
- [Apache Parquet](https://parquet.apache.org/documentation/latest/)
- [Google BigQuery](https://en.wikipedia.org/wiki/BigQuery)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from src.training import *

## CSV

Delimited files—most commonly comma-separated—are a very common data format for shared data.  Pandas' `.read_csv()` function has a huge number of optional parameters to control how a data file is read into a DataFrame.  One obvious and important one is `sep=<delimiter>` which defaults to comma but could be any other character.

Let us look at a few options against a small file we saw earlier.  Run `pd.read_csv?` in a cell to get a lot of documentation of parameters.

In [2]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols: 'list[HashableT] | Callable[[Hashable], bool] | None' = None,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters: 'Mapping[Hashable, Callable] | None' = None,
    true_values: 'list | None' = None,
    false_values: 'list | None' = None,
    skipinitialspace: 'bool' = False,
    skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values: 'Sequence[str] | Mapping[str, Sequence[str]] | None' = None,
    keep_default_na: 'bool' = True,
    n

While CSV is widely used and even reasonably fast to read, it does have some limitations.  Values within CSV or similar text formats are not typed directly in the encoding.  We need either to rely on our tool, e.g. Pandas, to infer types for us; or we need to specify them explicitly on import.

In [3]:
df = pd.read_csv('data/patient-records.csv')
df

,name,date,weight(kg),height(cm)
0,Alice,2011-01-01,85.1,170
1,Barb,2012-02-02,66.7,160
2,Carla,2013-03-03,29.5,120
3,Dagmar,2014-04-04,64.2,180


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        4 non-null      object 
 1   date        4 non-null      object 
 2   weight(kg)  4 non-null      float64
 3   height(cm)  4 non-null      int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 256.0+ bytes


In the `.info()` display, strings are generically Python objects.  For a person name, that is the best choice, but the date should be handled more specifically.

In [5]:
# Parse the date in the "date colum"
df = pd.read_csv('data/patient-records.csv', parse_dates=['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   name        4 non-null      object        
 1   date        4 non-null      datetime64[ns]
 2   weight(kg)  4 non-null      float64       
 3   height(cm)  4 non-null      int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 256.0+ bytes


In [6]:
# Change column names, explicit types of some columns, choose index
patients = pd.read_csv('data/patient-records.csv', 
                       skiprows=1,
                       names=['Patient', 'Visit_Date', 'Weight', 'Height'],
                       parse_dates=['Visit_Date'],
                       dtype={'Height': np.float16, 'Weight': np.float16},
                       index_col='Visit_Date')
patients

,Patient,Weight,Height
Visit_Date,,,
2011-01-01,Alice,85.1250,170.0
2012-02-02,Barb,66.6875,160.0
2013-03-03,Carla,29.5000,120.0
2014-04-04,Dagmar,64.1875,180.0


In [7]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4 entries, 2011-01-01 to 2014-04-04
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Patient  4 non-null      object 
 1   Weight   4 non-null      float16
 2   Height   4 non-null      float16
dtypes: float16(2), object(1)
memory usage: 80.0+ bytes


## Columns as Attributes

This simple DataFrame is an oportunity to note that if we happen to have column names that are valid Python identifiers, we can use a more compact attribute-style access rather than the dictionary-style indexing by column.  The square brackets are perfectly general, but the dot style sometimes looks nicer.

In [8]:
patients[(patients.Height > 165) & (patients.Weight < 80)]

,Patient,Weight,Height
Visit_Date,,,
2014-04-04,Dagmar,64.1875,180.0


In [9]:
patients[(patients['Height'] > 165) & (patients['Weight'] < 80)]

,Patient,Weight,Height
Visit_Date,,,
2014-04-04,Dagmar,64.1875,180.0


Once in a while you can get in trouble with the dot access because some attribute is both a column name and a DataFrame method or standard attribute.

In [10]:
df = pd.DataFrame({"A": [1, 5, 7], 'std': [31, 52, 68]})
df.A

0    1
1    5
2    7
Name: A, dtype: int64

In [11]:
df.std

<bound method DataFrame.std of    A  std
0  1   31
1  5   52
2  7   68>

In [12]:
df.std()

A       3.055050
std    18.556221
dtype: float64

In [13]:
df['std']

0    31
1    52
2    68
Name: std, dtype: int64

## Write a DataFrame Back to CSV

After we have manipulated DataFrames in some manner, we can easily write them to new delimited files.

In [14]:
import csv
patients.to_csv('tmp/patients.txt', sep='|', quoting=csv.QUOTE_NONNUMERIC)
!cat tmp/patients.txt

"Visit_Date"|"Patient"|"Weight"|"Height"
"2011-01-01"|"Alice"|85.125|170.0
"2012-02-02"|"Barb"|66.6875|160.0
"2013-03-03"|"Carla"|29.5|120.0
"2014-04-04"|"Dagmar"|64.1875|180.0


## SQL

There are a couple ways we can interact with SQL databases.  In particular, we can either read entire tables directly, or we can read the results of arbitrarily comple queries into DataFrames.  For the illustration here, an SQLite3 database is provided in the training materials.  However, the identical interfaces would work connecting to a distributed RDBMS (getting credentials correct for the connection object is a different topic, but once connection is established it is the same).

For reference, we first read in the Wisconsin breast cancer data set from CSV.

In [15]:
cancer_csv = pd.read_csv('data/wisconsin.csv')
cancer_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In the RDBMS we use, the data is broken out into separate tables for the features and the target.  In a more real world example, we would generally encounter many tables with a variety of foreign key constraints, triggers, indices, and so on.  Using some plain SQL (no Pandas), let us take a look at the schemata.

In [16]:
from pprint import pprint
import sqlite3
db = sqlite3.connect('data/wisconsin.db')
cur = db.cursor()
cur.execute("SELECT * FROM sqlite_master;")
pprint(cur.fetchall())
del db

[('table',
  'features',
  'features',
  2,
  'CREATE TABLE "features" (\n'
  '"Observation_No" INTEGER,\n'
  '  "mean_radius" REAL,\n'
  '  "mean_texture" REAL,\n'
  '  "mean_perimeter" REAL,\n'
  '  "mean_area" REAL,\n'
  '  "mean_smoothness" REAL,\n'
  '  "mean_compactness" REAL,\n'
  '  "mean_concavity" REAL,\n'
  '  "mean_concave_points" REAL,\n'
  '  "mean_symmetry" REAL,\n'
  '  "mean_fractal_dimension" REAL,\n'
  '  "radius_error" REAL,\n'
  '  "texture_error" REAL,\n'
  '  "perimeter_error" REAL,\n'
  '  "area_error" REAL,\n'
  '  "smoothness_error" REAL,\n'
  '  "compactness_error" REAL,\n'
  '  "concavity_error" REAL,\n'
  '  "concave_points_error" REAL,\n'
  '  "symmetry_error" REAL,\n'
  '  "fractal_dimension_error" REAL,\n'
  '  "worst_radius" REAL,\n'
  '  "worst_texture" REAL,\n'
  '  "worst_perimeter" REAL,\n'
  '  "worst_area" REAL,\n'
  '  "worst_smoothness" REAL,\n'
  '  "worst_compactness" REAL,\n'
  '  "worst_concavity" REAL,\n'
  '  "worst_concave_points" REAL,\n

### Entire table

Unfortunately, Pandas often relies on the SQLAlchemy *object relational mapper* (ORM) rather than use the raw database connection.  Usually the wrapper does no harm, but ORMs *do impose* their world view too much in much of their API (not the parts used here, however).

In [17]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/wisconsin.db')

In [18]:
cancer_features_sql = pd.read_sql_table('features', engine, 
                                        index_col='Observation_No')
cancer_features_sql

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,...,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
Observation_No,,,,,,,,,,,
0,17.99,10.38,122.80,1001.0,0.11840,...,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,...,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,...,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,...,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,...,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,...,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,...,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,...,0.30940,0.3403,0.1418,0.2218,0.07820


### Based on a query

We can issue an arbitrary query rather than read an entire table into a DataFrame.  If memory permits, it is often friendlier to do filtering and other operations within Pandas rather than within SQL; but this style allows whichever combination is most useful to you.

Note that `pd.read_sql()` is polymorphic in taking either a table name or a query, then doing some basic pattern matching to figure out which kind of request it is.  We find it better practice to be explicit about the query involved though.

In [19]:
sql = """
SELECT f.Observation_No, f.mean_radius, f.mean_texture, 
       f.mean_perimeter, t.benign 
FROM features f 
INNER JOIN target t 
WHERE f.Observation_No = t.Observation_No
LIMIT 100;
"""
cancer_with_target_sql = pd.read_sql_query(sql, engine, 
                                           index_col='Observation_No')
cancer_with_target_sql

,mean_radius,mean_texture,mean_perimeter,benign
Observation_No,,,,
0,17.990,10.38,122.80,0
1,20.570,17.77,132.90,0
2,19.690,21.25,130.00,0
3,11.420,20.38,77.58,0
4,20.290,14.34,135.10,0
...,...,...,...,...
95,20.260,23.03,132.40,0
96,12.180,17.84,77.79,1
97,9.787,19.94,62.11,1


## Write a DataFrame back to a table

After we have manipulated DataFrames in some manner, we can easily write them to tables in our RDBMS.

In [20]:
!rm tmp/cancer-partial.sqlite
db = sqlite3.connect('tmp/cancer-partial.sqlite')
cancer_with_target_sql.to_sql('my_table', db)

pprint(db.execute("SELECT * FROM my_table LIMIT 5").fetchall())

rm: cannot remove 'tmp/cancer-partial.sqlite': No such file or directory
[(0, 17.99, 10.38, 122.8, 0),
 (1, 20.57, 17.77, 132.9, 0),
 (2, 19.69, 21.25, 130.0, 0),
 (3, 11.42, 20.38, 77.58, 0),
 (4, 20.29, 14.34, 135.1, 0)]


## Excel Spreadsheets

Unfortunately, a great deal of data in the world lives in spreadsheets, and in Microsoft Excel specifically.  There are many things bad and broken about this format, but data comes as we get it.  For example:

* Very unreliable datatyping of columns and cells, especially if human entered data
* Poor handling of missing values
* Ragged, sparse, and irregular use of rectangular grid of rows/columns
* Computation expressed in distributed and hard-to-trace manner
* Inconsistent uses of tabs for hierarchy or structure
* Limitation on number of rows and inflexible datatypes
* Mediocre heuristics for inference of datatypes
* Use of columns for values of inconsistent datatypes
* **Extremely** slow compared to nearly every data format

Pandas allows us to read Excel sheets, and even to clean up the bad data we find once they are loaded into DataFrames.

In [21]:
cancer_excel = pd.read_excel('data/wisconsin.xlsx', 
                             sheet_name=['Features', 'Target'],
                             index_col='Observation_No')
cancer_excel['Features'].head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,...,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
Observation_No,,,,,,,,,,,
0,17.99,10.38,122.80,1001.0,0.11840,...,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,...,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,...,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,...,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,...,0.2050,0.4000,0.1625,0.2364,0.07678


In [22]:
cancer_excel['Target'].head()

,benign
Observation_No,
0,0
1,0
2,0
3,0
4,0


## External Data

Many of the `.read_<format>()` methods in Pandas are happy to accept URLs following a number of schemata (HTTP, HTTPS, FTP, S3, file, and others).

In [23]:
URL = ("https://bitbucket.org/davidmertz/sample-data/"
       "raw/9fb79b1e993e58ba7e3137f63726bb4a19bc0e8e/patient-records.csv")
pd.read_csv(URL)

,name,date,weight(kg),height(cm)
0,Alice,2011-01-01,85.1,170
1,Barb,2012-02-02,66.7,160
2,Carla,2013-03-03,29.5,120
3,Dagmar,2014-04-04,64.2,180


## JSON

Pandas DataFrames are inherently two dimensional grids of values.  This is very broadly useful, and maps well to formats like CSV, spreadsheets, and relational database tables or queries.  However, other data is hierarchical or has other structures.  

One of the most widely used hierarchical formats is Javascript Object Notation (JSON).  Flat grids are a subset, conceptually, of all possible nested structures.  But JSON is extermely widely used for data transmitted between web services, and for other purposes, so Pandas provides a number of ways to read/write JSON data that is "conceptually flat."

Some JSON formats result in lossy round-tripping with DataFrames since they do not represent all the components of a DataFrame.  Another thing to trip over is that JSON types by values; so e.g. if all values in a column are *equal to an integer* the writer will cast as integer.

In [24]:
df = pd.DataFrame({'col1': [1.0, 2, 3], 'col2': [5.1, 6.2, 7.3]},
                  index=['A', 'B', 'C'])
df.index.name = "Letter"
df

,col1,col2
Letter,,
A,1.0,5.1
B,2.0,6.2
C,3.0,7.3


In [25]:
style = 'split'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

'{"columns":["col1","col2"],"index":["A","B","C"],"data":[[1.0,5.1],[2.0,6.2],[3.0,7.3]]}'


,col1,col2
A,1,5.1
B,2,6.2
C,3,7.3


In [26]:
style = 'records'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

'[{"col1":1.0,"col2":5.1},{"col1":2.0,"col2":6.2},{"col1":3.0,"col2":7.3}]'


,col1,col2
0,1,5.1
1,2,6.2
2,3,7.3


In [27]:
style = 'index'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

'{"A":{"col1":1.0,"col2":5.1},"B":{"col1":2.0,"col2":6.2},"C":{"col1":3.0,"col2":7.3}}'


,col1,col2
A,1,5.1
B,2,6.2
C,3,7.3


In [28]:
style = 'columns'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

'{"col1":{"A":1.0,"B":2.0,"C":3.0},"col2":{"A":5.1,"B":6.2,"C":7.3}}'


,col1,col2
A,1,5.1
B,2,6.2
C,3,7.3


In [29]:
style = 'split'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

'{"columns":["col1","col2"],"index":["A","B","C"],"data":[[1.0,5.1],[2.0,6.2],[3.0,7.3]]}'


,col1,col2
A,1,5.1
B,2,6.2
C,3,7.3


In [30]:
style = 'table'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

'{"schema":{"fields":[{"name":"Letter","type":"string"},{"name":"col1","type":"number"},{"name":"col2","type":"number"}],"primaryKey":["Letter"],"pandas_version":"1.4.0"},"data":[{"Letter":"A","col1":1.0,"col2":5.1},{"Letter":"B","col1":2.0,"col2":6.2},{"Letter":"C","col1":3.0,"col2":7.3}]}'


,col1,col2
Letter,,
A,1.0,5.1
B,2.0,6.2
C,3.0,7.3


## Parquet and Arrow

Pandas can also read both the Parquet and Arrow formats, which are **far** higher performance than formats like Excel, CSV, JSON, and so on.  Both of these formats are column oriented and strictly data typed, which make them well suited to working on their data in data frames.

These two formats are slightly different creatures, however.  Arrow is a language-independent *memory format* so that varying libraries, across varying programming languages, can access the same data without performing copying.  For example, both Python Pandas and R dplyer can operate on the same memory, in principle.  Arrow also defines a serialization format, called Feather, which is essentially just a direct map of bytes in memory to bytes on disk.

Parquet is column-oriented *data file format* designed with very efficient data compression and encoding schemes.  However, when read into memory, Parquet data simply has the native in-memory layout of your data frame library. For modern versions of Pandas, that memory layout simply *is* Arrow.

In most cases, you should prefer to read and write Parquet rather than Arrow.  Its compression causes its files to use less disk space, and usually to read and write more quickly as well since CPU compression speeds greatly outpace disk write speeds.  Most of the same large range of tools, across many programming languages, support both Parquet and Arrow.

In [31]:
cancer_csv.to_parquet("tmp/wisconsin.parquet")
pd.read_parquet("tmp/wisconsin.parquet")

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,...,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign?
0,17.99,10.38,122.80,1001.0,0.11840,...,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,...,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,...,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,...,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,...,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,...,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,...,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,...,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,...,0.9387,0.2650,0.4087,0.12400,0


In [32]:
cancer_csv.to_feather("tmp/wisconsin.feather")
pd.read_feather("tmp/wisconsin.feather")

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,...,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign?
0,17.99,10.38,122.80,1001.0,0.11840,...,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,...,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,...,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,...,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,...,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,...,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,...,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,...,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,...,0.9387,0.2650,0.4087,0.12400,0


# Exercises

For a first few exercises, you will read and write from a variety of formats to get a feel for them.  Moreover, the speed of different formats can be dramatically different, so time operations against different formats (both reading and writing) using the `%timeit` magic in Jupyter.

Note that the `%timeit` macro creates its own namespace for operations, so when you want the actual result from an operation to work with more, you will need to run the function or method outside of the macro also.

## Obtain Data

Download the datasets provided as CSV URLs, and also save them locally as CSV.  Using the `tmp/` relative path within this training repository is a good choice of location to save files at.

* Is loading from local storage faster than from remote URL?
* What do you conclude about the main speed limitations in loading data from CSV?
* Does the same pattern hold for the smaller "Baby Names" dataset and the larger "Daily Temperatures" dataset?
* Were the inferred datatypes all the best choices for the respective fields?
* If you wanted to read some fields as different datatypes, how would you do that?
* What effect does file compression have on reading and writing CSV format?

In [33]:
# This dataset obtained from https://www.data.gov/
url = ("https://bitbucket.org/davidmertz/sample-data/raw/"
       "1bd7d3fbfc6842eb067ea9a9354b6b4e5b8597ab/Popular_Baby_Names.csv")
babynames = pd.read_csv(url)
babynames.to_csv('tmp/babynames.csv')
%timeit pd.read_csv(url)

767 ms ± 318 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The United States [National Oceanic and Atmospheric Administration (NOAA)](https://www.noaa.gov/) provides a number of useful datasets.  We will work with a subset of daily temperatures worldwide during 2019, which was obtained from https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/.  Notice that Pandas will seamlessly read compressed CSV files as well.

In [34]:
url = ("https://bitbucket.org/davidmertz/sample-data/raw/"
       "61872271984f66e3094c367cf90dfc4875a22e8d/NOAA-2019-partial.csv.gz")
temperatures = pd.read_csv(url)
%timeit pd.read_csv(url)

2.26 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Play with Different Formats

Save these datasets in other formats we have discussed, or even only mentioned in passing.  

* How does the speed of reading and writing compare?
* Do you lose any information round-tripping between various formats? (datatypes are information too)
* How do you expect performance to scale if you were to move from half-a-million rows to a hundred million rows?
* Would the answer vary across different formats?

In [ ]:
# Experiment here...

## Explain Advantages and Disadvantages

Explain to the person sitting next to you (or write in a cell below) what advantages and disadvantages you find in using different storage formats for Pandas DataFrames.  It is fair if one of the advantages is "my boss/work requires me to use X."

*Explanation of virtues and demerits of formats ...*

## Working with Data

Using ideas we have presented in passing and patterns you learned working with NumPy, try to extract only the temperatures for January, February, and March.

In [ ]:
# Yes, the solstice and equinox do not line up with months exactly
# If you would like to find the accurate cut-offs for "winter", please do
winter = ... # something using `temperatures`

Being data from the United States, temperatures are unfortunately provided in Fahrenheit. At least it would appear so:

In [ ]:
temperatures['TEMP'].max(), temperatures['TEMP'].min()

As mentioned, the data is a subset of the full year data. In later lessons we try to characterize the distributions of dates, longitudes, lattitudes, elevaations, and so on.  But, we do know:

> The official highest recorded temperature is now 56.7℃ (134℉), which was measured on 10 July 1913 at Greenland Ranch, Death Valley, California, USA.

So 92.2 would be quite low if the whole earth was represented as ℉.  But the number is vastly higher than any earth-surface ℃.

For extra credit, see if you can figure out how to add a new column to the `temperatures` DataFrame called `TEMP(C)` with the obvious meaning and correct values.

In [ ]:
# Add a column for Celcius temperatures
...

Write your improved data back to disk in your preferred data format.